# Entrega 2 - Introducción al Procesamiento del Lenguaje Natural 2018 


Este *notebook* de Python contiene las instrucciones para la segunda entrega del curso Introducción al Procesamiento de Lenguaje Natural. En el mismo se encontrará con instrucciones en bloques de texto y bloques de código para completar. Si bien no debe modificar la estructura base del notebook, puede agregar los bloques de texto o código que considere pertinentes para aportar claridad a la entrega.

Verifique que su entorno de Python 3 contiene todas las bibliotecas necesarias. Ejecute el bloque código a continuación para importar las bibliotecas nltk, sklearn y otras que le serán de utilidad. Verifique que se importan sin errores. 

In [1]:
import nltk
import sklearn
import os 
import json
import random

## Lectura de datos

### Corpus

Observe el corpus contenido en el directorio *restaurante-review-dataset* extraído de [1]. Note la partición en entrenamiento (train), validación (val) y evaluación (test). Ejecute el bloque a continuación para cargar el contenido del corpus en tres variable: *train*, *validation* y *test*. Se utiliza como estructura de datos una lista de pares (comentario, valor), donde comentario es una string con el comentario y valor corresponde a la string "POS" o "NEG" si el comentario es positivo o negativo, respectivamente.
 
[1] Dubiau, L., & Ale, J. M. (2013). Análisis de Sentimientos sobre un Corpus en Español: Experimentación con un Caso de Estudio. In Proceedings of the 14th Argentine Symposium on Artificial Intelligence, ASAI (pp. 36-47).


In [2]:
def load_data(directory, val):
    data = []
    for file in os.listdir(directory):
        with open(os.path.join(directory, file)) as f:
            file_data = json.load(f)
            data += [(l,val) for l in file_data]
    return data

corpus_train_dirs = [
    ("./restaurante-review-dataset/train-neg/", "NEG"),
    ("./restaurante-review-dataset/train-pos/", "POS"),
]
corpus_val_dirs = [
    ("./restaurante-review-dataset/val-neg/", "NEG"),
    ("./restaurante-review-dataset/val-pos/", "POS"),
]
corpus_test_dirs = [
    ("./restaurante-review-dataset/test-neg/", "NEG"),
    ("./restaurante-review-dataset/test-pos/", "POS"),
]

train, validation, test = [],[],[]
for d,v in corpus_train_dirs:
    train += load_data(d, v)
for d,v in corpus_val_dirs:
    validation += load_data(d, v)
for d,v in corpus_test_dirs:
    test += load_data(d, v)

random.Random(1234).shuffle(train)
random.Random(2345).shuffle(validation)
random.Random(3456).shuffle(test)

Despliegue en pantalla la cantidad de elementos positivos, negativos y totales de cada partición del corpus.

In [3]:
trainPOS = sum(1 for comentario in train if comentario[1] == 'POS')
trainTOT = len(train)
trainNEG = trainTOT - trainPOS

validationPOS = sum(1 for comentario in validation if comentario[1] == 'POS')
validationTOT = len(validation)
validationNEG = validationTOT - validationPOS

testPOS = sum(1 for comentario in test if comentario[1] == 'POS')
testTOT = len(test)
testNEG = testTOT - testPOS

print("En la partición 'train' hay un total de {} comentarios de los cuales {} son positivos y {} son negativos.\n".format(trainTOT, trainPOS, trainNEG))
print("En la partición 'validation' hay un total de {} comentarios de los cuales {} son positivos y {} son negativos.\n".format(validationTOT, validationPOS, validationNEG))
print("En la partición 'test' hay un total de {} comentarios de los cuales {} son positivos y {} son negativos.\n".format(testTOT, testPOS, testNEG))

En la partición 'train' hay un total de 34506 comentarios de los cuales 23114 son positivos y 11392 son negativos.

En la partición 'validation' hay un total de 8587 comentarios de los cuales 5582 son positivos y 3005 son negativos.

En la partición 'test' hay un total de 9348 comentarios de los cuales 6112 son positivos y 3236 son negativos.



### Vectores

Los *word vectors* son representaciones vectoriales de las palabras construidas a partir de grandes colecciones de texto. Junto a este *notebook* se imparte un reportorio de vectores construido a partir de un corpus en español usando *skip-gram* con *negative sampling* (SGNS)[1]

En esta sección cargará en memoria y utilizará el repertorio de vectores impartido. Además, se evaluará la cobertura del repertorio en el corpus y se estudiará las palabras del corpus no contempladas en el repertorio de vectores (*out-of-vocabulary terms*).

En el directorio *vectores* se encuentran dos archivos:

- sgns_spvectors_300.txt con la lista de palabras del repertorio
- sgns_spvectors_300.npy con la matriz que contiene cada vector

Defina un mecanismo para cargar en memoria los vectores y para obtener en tiempo eficiente el vector correspondiente a una palabra. Tenga en cuenta lo siguiente:

- puede serle útil un diccionario que a cada palabra le corresponda su índice en la matriz
- resuelva que hacer con las palabras que no están en el repertorio de vectores


[1] Mikolov, T., Sutskever, I., Chen, K., Corrado, G. S., & Dean, J. (2013). Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems (pp. 3111-3119).


In [4]:
import numpy as np

vectors = np.load('./vectores/sgns_spvectors_300.npy')

with open('./vectores/sgns_spvectors_300.txt', 'r') as f:
    words = f.read().splitlines()

_, totalColumnsVectors = vectors.shape

numToWord = dict(list(enumerate(words)))
wordToNum = {v: k for k, v in numToWord.items()}

def getVector(vectors, word):
    try:
        vec = vectors[wordToNum[word]]
    except KeyError:
        return [0 for _ in range(totalColumnsVectors)]
    return vec

A continuación realice pruebas con los vectores almacenados. En el siguiente bloque de código realice lo siguiente:

1. Defina una función de similitud entre vectores
2. Imprima la similitud entre los vectores de las palabras de ejemplo
3. Defina un conjunto de pares de palabras del repertorio de vectores (*pares_estudiante*)
4. Imprima la similitud de los vectores de las palabras definidas en el paso anterior
5. Imprima el vector correspondiente a una palabra que no se encuentre en el repertorio de vectores

In [5]:
from scipy.spatial.distance import cosine
from numpy.linalg import norm

def similarity(vector1, vector2):
    if ((norm(vector1) == 0) or (norm(vector2) == 0)):
        return 0
    return 1 - cosine(vector1, vector2)

pares = [
    ('bueno','excelente'),
    ('bueno','buena'),
    ('bueno','malo'),
    ('malo','espantoso'),
    ('comida', 'ambiente'),
    ('comida', 'bebida'),
    ('comida', 'postre'),
    ('comida', 'sabor'),
    ('servicio', 'comida'),
    ('servicio', 'ambiente'),
    ('ambiente', 'calor'),
    ('frío', 'calor'),
]

print("Conjunto 'pares':\n")

for pair in pares:
    vector0 = getVector(vectors, pair[0])
    vector1 = getVector(vectors, pair[1])
    print("La similitud entre '{}' y '{}' es {}.".format(pair[0], pair[1], similarity(vector0, vector1)))

print("\nConjunto 'pares_estudiante':\n")

pares_estudiante = [
    ('bodka', 'vodka'),
    ('manuela', 'marea'),
    ('salado', 'dulce'),
    ('rico', 'sabroso'),
    ('agua', 'salada'),
    ('postre', 'dulce'),
    ('ambiente', 'agradable'),
    ('tecnologia', 'idem'),
    ('pista', 'carreras'),
    ('administración', 'empresas'),
    ('anterior', 'posterior'),
    ('buenos', 'aires'),
    ('francia', 'españa'),
    ('francia', 'espana'),
    ('france', 'spain'),
    ('alejandro', 'magno'),
    ('año', 'nuevo'),
    ('caño', 'caños'),
    ('el', 'la'),
    ('eso', 'esa'),
] 

for pair in pares_estudiante:
    vector0 = getVector(vectors, pair[0])
    vector1 = getVector(vectors, pair[1])
    print("La similitud entre '{}' y '{}' es {}.".format(pair[0], pair[1], similarity(vector0, vector1)))
    
print("\nEl vector correspondiente a una palabra que no se encuentre en el repertorio de vectores es: \n", getVector(vectors, 'palabrafueradelrepertorio'))

Conjunto 'pares':

La similitud entre 'bueno' y 'excelente' es 0.44070714712142944.
La similitud entre 'bueno' y 'buena' es 0.5369178056716919.
La similitud entre 'bueno' y 'malo' es 0.7269262075424194.
La similitud entre 'malo' y 'espantoso' es 0.5346347093582153.
La similitud entre 'comida' y 'ambiente' es 0.23836146295070648.
La similitud entre 'comida' y 'bebida' es 0.6403059959411621.
La similitud entre 'comida' y 'postre' es 0.46853193640708923.
La similitud entre 'comida' y 'sabor' es 0.44875413179397583.
La similitud entre 'servicio' y 'comida' es 0.2339814007282257.
La similitud entre 'servicio' y 'ambiente' es 0.22486859560012817.
La similitud entre 'ambiente' y 'calor' es 0.3984523117542267.
La similitud entre 'frío' y 'calor' es 0.7722591757774353.

Conjunto 'pares_estudiante':

La similitud entre 'bodka' y 'vodka' es 0.5515549182891846.
La similitud entre 'manuela' y 'marea' es 0.034327760338783264.
La similitud entre 'salado' y 'dulce' es 0.5086449384689331.
La similitud 

¿Qué observa en los resultados obtenidos?

**Respuesta:**

Para analizar los resultados obtenidos debemos tener en cuenta el funcionamiento del modelo skip-gram. Dicho modelo es implementado como una red neuronal de una única capa oculta que tiene como entrada un vector 'one-hot' (esto es, un vector de largo _n = cantidad de tokens en el corpus_ y con valor 1 en la posición que corresponde a la palabra ingresada y 0 en los demás valores) que representa una palabra, devolviendo como salida un vector de probabilidades de que cada palabra del corpus de entrenamiento se encuentre en el contexto de la palabra ingresada (o sea, en su proximidad). Luego, los vectores se obtienen de la matriz resultante en la capa oculta de la red. Este modelo forma parte del conjunto de modelos utilizados en _word2vec_.
De este modelo se deriva que la distancia entre dos palabras en su representación vectorial nos permite obtener una medida de si una de ellas aparece en contextos iguales a la otra en nuestro corpus de entrenamiento.

Es así que podemos observar, por ejemplo, que las palabras 'bueno' y 'malo' aparecen en el mismo contexto varias veces ya que tienen un valor de similitud alto. Lo mismo ocurre con palabras como 'frío' y 'calor', 'francia' y 'españa' mientras que palabras como 'manuela' y 'marea' no aparecen prácticamente nunca en el mismo contexto. Las palabras 'buenos' y 'aires' son de las que mayor valor de similitud tienen, ya que suelen aparecer juntas conformando un nombre propio. Parece ser que no se hizo en este caso, pero en [1] recomiendan tratar este tipo de "frases" como una palabra en sí, manteniendo separado el significado de las palabras cuando se encuentran aparte de cuando se encuentran juntas.
También podemos ver que pronombres con el mismo significado pero que varían en género tienen una similitud muy alta, lo cual es previsible.

Cabe destacar ciertos casos en los que podría suceder que intuitivamente se piense que la similitud debería rondar cierto valor (a nuestro juicio), pero que no resulta así. Tales son los casos como 'francia' y 'espana' (esta última es escrita así cuando el escritor tiene un teclado en inglés), 'pista' y 'carreras' o 'rico' y 'sabroso'. Esto puede suceder porque las palabras no ocurrían en el mismo contexto lo suficiente como para impactar en el valor de similitud. Otro causante de esto es que la ventana de contexto utilizada (esto es, cuantas palabras se consideran hacia la derecha y hacia la izquierda de la palabra objetivo) no tenga un tamaño suficiente para recoger ciertas palabras en el contexto de la palabra objetivo, o también que la ventana no sea simétrica (por ejemplo, considera 8 palabras antes de la objetivo y 1 palabra luego de la misma). 

En el bloque de código a continuación realice lo siguiente:
- Separe el corpus en palabras con nltk.wordpunct_tokenize (todas la partes: train, validacion y test)
- Convierta las palabras a minúsculas dado que el repertorio de vectores está en minúsculas
- Almacene las palabras resultantes en una variable llamada *vocabulario*. Considere una estructura adecuada para no tener palabras repetidas.
- Despliegue en pantalla la cantidad de palabras de *vocabulario*

In [6]:
vocabulario = set()

for comment in train + validation + test:
    tokenizedComment = nltk.wordpunct_tokenize(comment[0].lower())
    for word in tokenizedComment:
        vocabulario.add(word)

print("El vocabulario tiene " + str(len(vocabulario)) + " palabras.")


El vocabulario tiene 53489 palabras.


Construya los siguientes dos conjuntos:

1. Palabras de *vocabulario* que tienen un vector asociado en el repertorio de vectores.
2. Palabras de *vocabulario* que **no** tienen un vector en el repertorio de vectores.

Imprima la cantidad de palabras de cada conjunto. Imprima además un muestreo de las palabras del conjunto 2.



In [7]:
hasVector = set()
hasVectorNot = set()

for word in vocabulario:
    try:
        vec = vectors[wordToNum[word]]
    except KeyError:
        hasVectorNot.add(word)
    hasVector.add(word)
    
print("Cantidad de palabras que tienen un vector asociado: {}".format(len(hasVector)))
print("Cantidad de palabras que NO tienen un vector asociado: {}\n".format(len(hasVectorNot)))

for word in enumerate(list(hasVectorNot)):
    print(word[1])
    if (word[0] == 40): break;
    


Cantidad de palabras que tienen un vector asociado: 53489
Cantidad de palabras que NO tienen un vector asociado: 9278

dorsia
resturant
malisimos
infaltablr
agregarona
browny
favorrrrrrr
tobanyaki
atendiö
rneado
desgustacion
bocconotto
agrergaron
morsilla
torillas
recomendóel
!!!!..
rools
diasculpas
pmadero
cumpliamos
ambientaón
pavè
recocinados
marucuyá
histéricamante
caipirvoska
:),
sweetbreads
sacarn
ccaro
hugoheadrecomiendo
postrecon
negativisima
excursao
nautural
tandoris
gamberetti
chopeada
deshabridos
bueniiiiisimos


¿Qué observa en el muestreo de palabras que no tienen un vector asociado (conjunto 2)?


**Respuesta:**

Las 20 palabras impresas son en su mayoría palabras que tienen errores ortográficos o "typos" (abreviatura de _typographical error_ en español "error tipográfico"). Otras palabras que también aparecen en el muestreo son emoticones (':)'), algunas puntuaciones "extrañas" ('!!!!..') y palabras que son escritas haciendo énfasis en algun aspecto (como 'bueniiiiisimos' o 'favorrrrrrr') , como también algunas que, aunque no son tan raras, simplemente pueden no aparecen en el corpus de entrenamiento (como 'resturant' o 'histéricamante'). También aparecen palabras que son heredadas de otros idiomas (como 'chopeada', 'browny' o 'sweetbreads').

Esto es un suceso común, ya que los comentarios son escritos por personas de todo estrato social y edad, lo que lleva a que se encuentren palabras con faltas ortográficas o "raras". Además, es común que varias letras se intercambien por otras cuando se escribe en el teclado ("typo").
También se debe considerar que los lenguajes se encuentran en constante evolución, y que dependiendo de los diferentes tiempos de donde se comparen texto, se pueden encontrar palabras que no existían en el espacio temporal de uno pero si en otro.
Otro factor a tener en cuenta es que los corpus no traten sobre el mismo tema, provocando que palabras del lenguaje específico de los comentarios de restaurantes no se encuentren en el corpus.

## Representación vectorial de la oración

### Bolsa de Palabras

Realice una representación de bolsa de palabras con *stemming* para los comentarios del corpus considerando únicamente los conjuntos de entrenamiento y validación. Utilice la clase *sklearn.CountVectorizer* con una configuración de parámetros con considere adecuada. Esta representación será utilizada posteriormente para realizar clasificiación supervisada.

**Sugerencia:** Utilice el parámetro *min_df* y *max_df* para reducir la dimensión del vector de la bolsa de palabras. Se sugiere que la dimensión de la representación sea menor a 500.

In [8]:
from nltk.stem.snowball import SpanishStemmer
from copy import copy

spanish_stemmer = SpanishStemmer()

tokenRegex = '[a-záéíóúñ]{3,}'
transf = sklearn.feature_extraction.text.CountVectorizer(max_df=0.2, min_df=400, ngram_range=(1,1), analyzer='word', 
                                                         lowercase=True, token_pattern=tokenRegex, stop_words=None)

# Se define una nueva función que servira como nuevo parametro tokenizer del constructor CountVectorizer
def stem_tokenizer(text):
    tokenizer = transf.build_tokenizer()
    return [spanish_stemmer.stem(word) for word in tokenizer(text)]

trainCom = [comentario for (comentario, valor) in train]
validationCom = [comentario for (comentario, valor) in validation]
comments = trainCom + validationCom

transf_with_stem = copy(transf).set_params(tokenizer=stem_tokenizer)
transf_with_stem.fit(comments)

vectores_with_stem = transf_with_stem.transform(comments)

En el siguiente bloque de código realice lo siguiente:

- Despliegue la representacion (bow) de la oración presentada como ejemplo
- Despliegue además la cantidad de palabras de la bolsa de palabras

In [9]:
oracion_ej = 'Muy pero muy buena rica la comida y muy ricas tartas'

# COMPLETE A PARTIR DE AQUI
bow = transf_with_stem.transform([oracion_ej])
print("La representación de la oración es: " + str(bow))
diccionario_with_stem = transf_with_stem.get_feature_names()
print("La bolsa de palabras tiene " + str(len(diccionario_with_stem)) + " palabras.")
#print(vectores_with_stem)

La representación de la oración es:   (0, 386)	2
La bolsa de palabras tiene 486 palabras.


Ejecute el bloque de código a continuación para definir la funcion *imprimir_tiempo*. Esta función despliega en pantalla el tiempo transcurrido a partir del timestamp pasado como parámetro. Si desea puede utilizarla para medir el tiempo de sus ejecuciones.

In [10]:
import time 

def imprimir_tiempo(ts):
    print("--- %s mins ---" % (float(time.time() - ts)/60))

Implemente la función *data2Xy_bow* cuyo encabezado se presenta en el bloque de código a continuación. La función transforma en vectores a los comentarios con la bolsa de palabras almacenándolos en X y sus etiquetas en y.

In [11]:
def data2Xy_bow(bow, data):
    X = bow.transform([comentario for (comentario, etiqueta) in data])
    y = [etiqueta for (comentario,etiqueta) in data]
    return X,y


Considere la lista de pares de comentarios *lista_comentarios*. En la lista *lista_comentarios_estudiante* escriba pares de comentarios que considere pertinentes para ver su similitud según el *bow* definido. Imprima en pantalla los comentarios de ambas listas junto a la similaridad obtenida según la representación de *bow*.

In [12]:
lista_comentarios = [
    ('muy rica la comida, buenas pizzas', 'excelente pizza la salsa estaba muy rica'),
    ('no me gustó para nada. mala atención', 'me pareció todo bastante malo'),
    ('que buen servicio, hay que volver', 'excelente todo, me verán seguido por ahí'),
    ('las tartas no me gustaron', 'muy bueno servicio'),
]

lista_comentarios_estudiante = [
    ('Es todo muy muy rico y de alta calidad!', 'muy muy rico todo. servicios excellente. buen curry.'),
    ('Guijón: Por favor no cambien nunca. Gracias', 'caro y malo no volveria nunca mas'),
    ('Horrible. Muy mala comida y carísimo.', 'carisimo, la comida horrible!!!!!!!!!!!!'),
    ('En los rubros ambiente, atención y comida, lo mejor de Palermo.', 'mala atencion y falta de buena comida y ambiente, cambien de rubro.')
]

# COMPLETE A PARTIR DE AQUI
for par in lista_comentarios + lista_comentarios_estudiante:
    vs = transf_with_stem.transform(par)
    print("La similitud entre '{}' y '{}' es {}.".format(par[0], par[1], similarity(vs.toarray()[0], vs.toarray()[1])))


La similitud entre 'muy rica la comida, buenas pizzas' y 'excelente pizza la salsa estaba muy rica' es 0.7071067811865476.
La similitud entre 'no me gustó para nada. mala atención' y 'me pareció todo bastante malo' es 0.33333333333333337.
La similitud entre 'que buen servicio, hay que volver' y 'excelente todo, me verán seguido por ahí' es 0.0.
La similitud entre 'las tartas no me gustaron' y 'muy bueno servicio' es 0.0.
La similitud entre 'Es todo muy muy rico y de alta calidad!' y 'muy muy rico todo. servicios excellente. buen curry.' es 0.5.
La similitud entre 'Guijón: Por favor no cambien nunca. Gracias' y 'caro y malo no volveria nunca mas' es 0.25.
La similitud entre 'Horrible. Muy mala comida y carísimo.' y 'carisimo, la comida horrible!!!!!!!!!!!!' es 0.816496580927726.
La similitud entre 'En los rubros ambiente, atención y comida, lo mejor de Palermo.' y 'mala atencion y falta de buena comida y ambiente, cambien de rubro.' es 0.0.


### Centroide de vectores

En esta parte se representará cada comentario como el centroide de los vectores de las palabras que lo forman. Se pide implementar la función *txt2vec* que dado un comentario y el repertorio de vectores calcula el promedio de los vectores de las palabras del comentario.

In [13]:
def txt2vec(vectores, comentario):
    comment_words = nltk.wordpunct_tokenize(comentario.lower())       
    vectores_comentario = [getVector(vectores, word) for word in comment_words] 
            
    if (len(vectores_comentario) == 0):
        return np.zeros(totalColumnsVectors)
    
    return np.mean(vectores_comentario, axis=0)

Implemente la función *data2Xy_vec* cuyo encabezado se presenta en el bloque de código a continuación. La función transforma a los comentarios en su representación de centroide de vectores (las filas de X) y sus etiquetas son las componente del vector y.

In [14]:
def data2Xy_vec(vec, data):        
    X = [txt2vec(vec, comentario) for (comentario, etiqueta) in data]
    y = [etiqueta for (comentario, etiqueta) in data]
    return X, y

Despliegue en pantalla los pares de comentarios de *lista_comentarios* y *lista_comentarios_estudiante* junto con la similitud de sus representaciones de centroide de vectores.

In [15]:
def similitud_centroides(c1, c2):
    centroides_palabras_c1 = txt2vec(vectors, c1)
    centroides_palabras_c2 = txt2vec(vectors, c2)
    
    return similarity(centroides_palabras_c1, centroides_palabras_c2)

for c1,c2 in lista_comentarios + lista_comentarios_estudiante:    
    similitud_par_comentarios = similitud_centroides(c1, c2)
    print("La similitud entre '{}' y '{}' es {}.".format(c1, c2, similitud_par_comentarios))

La similitud entre 'muy rica la comida, buenas pizzas' y 'excelente pizza la salsa estaba muy rica' es 0.9032563488394832.
La similitud entre 'no me gustó para nada. mala atención' y 'me pareció todo bastante malo' es 0.8814249422516375.
La similitud entre 'que buen servicio, hay que volver' y 'excelente todo, me verán seguido por ahí' es 0.8332775016069546.
La similitud entre 'las tartas no me gustaron' y 'muy bueno servicio' es 0.6773157715797424.
La similitud entre 'Es todo muy muy rico y de alta calidad!' y 'muy muy rico todo. servicios excellente. buen curry.' es 0.877984942326619.
La similitud entre 'Guijón: Por favor no cambien nunca. Gracias' y 'caro y malo no volveria nunca mas' es 0.8121591779415548.
La similitud entre 'Horrible. Muy mala comida y carísimo.' y 'carisimo, la comida horrible!!!!!!!!!!!!' es 0.867017408240695.
La similitud entre 'En los rubros ambiente, atención y comida, lo mejor de Palermo.' y 'mala atencion y falta de buena comida y ambiente, cambien de rubro

Comente los resultados de similitud de oraciones obtenidos con las representaciones de bolsa de palabras y las de centroide de vectores.

**Respuesta:**

En general las similitudes obtenidas con bolsa de palabras son más bajas debido a que solo se miran palabras cuya raíz coincida con la raíz de palabras en el otro comentario, y no necesariamente la similitud (semántica) de dos comentarios implica que se compartan palabras, e incluso comentarios relativamente parecidos dan una similitud de 0.
En cambio con centroide de vectores se logra detectar similitudes semánticas más allá de tener o no las mismas palabras, pero a su vez puede suceder que oraciones muy diferentes tengan centroides muy cercanos entre sí debido a meras casualidades que pudieron ocurrir en el corpus con el que se construyeron los vectores.

## Análisis de Sentimiento

### Support Vector Machine

En esta sección entrene un clasificador *SVM* de *sklearn* para ambas representaciones (centroide de vectores y bolsa de palabras). Busque una configuración de hiperparámetros adecuada utilizando como referencia el conjunto de validación. Utilice para comparar resultados la medida *accuracy*.

In [16]:
X,y = data2Xy_bow(transf_with_stem, train)
clf = sklearn.svm.LinearSVC(C=22, max_iter=100000)
ti = time.time()
clf.fit(X, y)
print("Tiempo de entrenamiento: \n")
imprimir_tiempo(ti)

Tiempo de entrenamiento: 

--- 2.4706937154134114 mins ---


In [17]:
Xval, yval = data2Xy_bow(transf_with_stem, validation)
ti = time.time()
print("Accuracy: " + str(clf.score(Xval, yval)))
print("Tiempo de validación: \n")
imprimir_tiempo(ti)

Accuracy: 0.9425876324676837
Tiempo de validación: 

--- 8.800824483235677e-05 mins ---


In [18]:
X_centroide_train,y_centroide_train = data2Xy_vec(vectors, train)

SVM_centroide = sklearn.svm.LinearSVC(C=22, max_iter=10000)
ti = time.time()
SVM_centroide.fit(X_centroide_train, y_centroide_train)
print("Tiempo de entrenamiento: \n")
imprimir_tiempo(ti)

Tiempo de entrenamiento: 

--- 0.2703379432360331 mins ---


In [19]:
X_centroide_val, y_centroide_val = data2Xy_vec(vectors, validation)
ti = time.time()
print("Accuracy: " + str(SVM_centroide.score(X_centroide_val, y_centroide_val)))
print("Tiempo de validación: \n")
imprimir_tiempo(ti)

Accuracy: 0.9559799697216723
Tiempo de validación: 

--- 0.0003843824068705241 mins ---


Despliegue los resultados obtenidos con *SVM* para ambas representaciones.

In [20]:
def printClassifierResult(classifier, X, y, modelText):
    predicted = classifier.predict(X)
    conf = sklearn.metrics.confusion_matrix(y, predicted)
    tp, tn, fp, fn = conf[1][1], conf[0][0], conf[0][1], conf[1][0]
    print("\nPara el modelo de {} se obtiene lo siguiente:".format(modelText))
    print("Hay {} verdaderos positivos, {} verdaderos negativos, {} falsos positivos, {} falsos negativos".format(tp, tn, fp, fn))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1Score = 2 * precision * recall / (precision + recall)
    print("Precision: {}, recall: {}, Fscore: {}".format(precision, recall, f1Score))

In [21]:
printClassifierResult(clf, Xval, yval, "SVM con Bag of Words")
printClassifierResult(SVM_centroide, X_centroide_val, y_centroide_val, "SVM con Centroides")


Para el modelo de SVM con Bag of Words se obtiene lo siguiente:
Hay 5453 verdaderos positivos, 2641 verdaderos negativos, 364 falsos positivos, 129 falsos negativos
Precision: 0.937424789410349, recall: 0.9768900035829452, Fscore: 0.9567505921572068

Para el modelo de SVM con Centroides se obtiene lo siguiente:
Hay 5437 verdaderos positivos, 2772 verdaderos negativos, 233 falsos positivos, 145 falsos negativos
Precision: 0.9589065255731922, recall: 0.9740236474381943, Fscore: 0.9664059722715962


### Feed Fowrward Neural Networks

En esta sección utilizará un clasificador de red neuronal *feed forward* (*multilayer perceptron* - MLP) para realizar la clasificación de sentimiento de los comentarios considerando ambas representaciones: centroide de vectores y bolsa de palabras. Busque una configuración de hiperparámetros adecuada tomando como referencia la medida de *accuracy* obtenida en el conjunto de validación. 

In [22]:
from sklearn.neural_network import MLPClassifier
clfNN = MLPClassifier(solver='adam', alpha=1e-5, batch_size=500, max_iter=500, learning_rate='adaptive', hidden_layer_sizes=(5, 2), shuffle=False, activation='identity', tol=1e-4)
ti = time.time()
clfNN.fit(X,y)
print("Tiempo de entrenamiento: \n")
imprimir_tiempo(ti)

ti = time.time()
print("\nAccuracy: " + str(clfNN.score(Xval,yval)))
print("Tiempo de validación: \n")
imprimir_tiempo(ti)

Tiempo de entrenamiento: 

--- 0.06974427700042725 mins ---

Accuracy: 0.9430534528939094
Tiempo de validación: 

--- 0.00015703439712524414 mins ---


In [23]:
from sklearn.neural_network import MLPClassifier
NN_centroide = MLPClassifier(solver='adam', alpha=1e-10, batch_size=500, max_iter=500, learning_rate='adaptive', hidden_layer_sizes=(5, 2), shuffle=False, activation='identity', tol=1e-4)
ti = time.time()
NN_centroide.fit(X_centroide_train, y_centroide_train)
print("Tiempo de entrenamiento: \n")
imprimir_tiempo(ti)

ti = time.time()
print("\nAccuracy: " + str(NN_centroide.score(X_centroide_val, y_centroide_val)))
print("Tiempo de validación: \n")
imprimir_tiempo(ti)

Tiempo de entrenamiento: 

--- 0.5119045337041219 mins ---

Accuracy: 0.9529521369512053
Tiempo de validación: 

--- 0.0005107919375101725 mins ---


Despliegue los mejores resultados obtenidos para ambas representaciones.

In [24]:
printClassifierResult(clfNN, Xval, yval, "Feed Forward Neural Network con Bag of Words")
printClassifierResult(NN_centroide, X_centroide_val, y_centroide_val, "Feed Forward Neural Network con Centroides")


Para el modelo de Feed Forward Neural Network con Bag of Words se obtiene lo siguiente:
Hay 5440 verdaderos positivos, 2658 verdaderos negativos, 347 falsos positivos, 142 falsos negativos
Precision: 0.940038016243304, recall: 0.974561089215335, Fscore: 0.9569883015216817

Para el modelo de Feed Forward Neural Network con Centroides se obtiene lo siguiente:
Hay 5490 verdaderos positivos, 2693 verdaderos negativos, 312 falsos positivos, 92 falsos negativos
Precision: 0.9462254395036195, recall: 0.9835184521676819, Fscore: 0.9645115952213633


### Mejores Resultados en Validación

Analice los resultados obtenidos con cada clasificador y cada representación en el conjunto de validación. 

Tenga en cuenta que debe considerar al menos los siguientes 4 clasificadores:

- SVM con BOW
- SVM con Centroide
- MLP con BOW
- MLP con Centroide

Realice los comentarios que considere adecuados respecto a la comparación y resultados obtenidos. Si lo desea puede agregar bloques de código que muestren resultados adicionales.

**Respuesta:** 
###### SVM:

En el caso de SVM, se probaron dos implementaciones del clasificador SVM: LinearSVC y SVC. En cuanto a la comparación entre resultados obtenidos, son similares en las dos implementaciones. LinearSVC se comportó mejor utilizando la representación de centroides y en el caso de SVC se obtuvo una pequeña mejora utilizando la representación de bolsa de palabras. En relación a la performance computacional, la segunda resulta demorar menos. Esto puede suceder por las diferencias de implementación entre las dos funcionalidades o por desactivar el shrinking para mejorar los resultados (lo cual cuando se activa mejora los tiempos). 

Respecto a la comparación entre los resultados obtenidos para las dos representaciones vectoriales de palabras, si tomamos el valor F (_Fscore_) como medida de correctitud general, se ve que el clasificador SVM con representación de centroide mejora levemente al clasificador SVM con representación de bolsa de palabras. Esto puede deberse a que la representación de centroides recoge aspectos estadísticos a un nivel más global que el caso de bag of words, para la cual se consideran menos palabras.
El otro aspecto a evaluar es que se percibe una mejora significativa en el tiempo de entrenamiento y validación para el caso en que se utiliza la representación de centroides. Esto puede suceder debido a que los vectores de dicha representación tienen una menor cantidad de features que los que se obtienen utilizando la representación de bolsa de palabras (300 en representación de centroides, 486 en bolsa de palabras).

###### MLP:

En el caso de MLP, se probó una única implementación, MLPClassifier. Vemos que al igual que con LinearSVC, la representación de centroides mejora la precision de la red neuronal y mantiene un recall relativamente parecido a cuando se utiliza la representación de bolsa de palabras. Se ajustó el parámetro alfa de regularización de tal forma que se produjeran los mejores resultados para cada representación. Esta puede ser una de las razones, además de las mencionadas antes, por la cual se da una diferencia en el tiempo de ejecución requerido para entrenar cada clasificador, ya que cuanto menor es alfa más se intenta ajustar la función.

###### SVM vs. MLP:

El primer factor que salta a la vista es la diferencia de performance de tiempo computacional. El clasificador que utiliza la red neuronal posee una amplia mejora (tanto en tiempo de entrenamiento como de validación) con respecto a los dos clasificadores que utilizan SVM.

Prosiguiendo con los resultados en la validación se puede ver que en el caso del clasificador MLP, presenta un valor F similar al clasificador SVM para el caso de la representación con centroides (considerando la implementación LinearSVC), y apenas menor para el caso de la representación de bolsa de palabras (considerando la implementación SVC).

Yendo a los resultados más concretos, el clasificador LinearSVC obtiene los mejores valores con respecto a la cantidad de ejemplos clasificados correctamente. En relación a los ejemplos clasificados incorrectamente, se obtienen resultados mixtos: el clasificador MLP empeora la cantidad de falsos positivos en relación con LinearSVC (no así con SVC) cuando se utiliza la representación de centroides, pero reduce la cantidad de falsos negativos en relación a las 2 implementaciones. Para el caso de la representación de comentarios con bolsa de palabras, MLP empeora la cantidad de falsos negativos y en cuanto a los falsos positivos, se obtiene el mejor resultado para SVC, seguido por el clasificador MLP y, por último, el clasificador LinearSVC.

Podemos ver estos resultados proyectados en una **precision** máxima obtenida para LinearSVC con centroides (y con un poco menos MLP con centroides) debido a la alta cantidad de ejemplos clasificados correctamente. En cuanto a la **recall**, la máxima se obtuvo para MLP con centroides. Esto nos indica que los mejores resultados se obtuvieron con LinearSVC y MLP, ambos utilizando la representación con centroides.

Podemos concluir entonces que, para el corpus de comentarios, aunque los dos clasificadores (y las dos implementaciones del clasificador SVM) arrojan resultados similares para ambas representaciones de comentarios, el clasificador MLP presenta una mejora en cuanto a tiempo computacional y en la calidad de resultados en general que puede inclinar la balanza a su favor.

**Clasificador SVM implementado en SVC**

In [25]:
X,y = data2Xy_bow(transf_with_stem, train)
clf2 = sklearn.svm.SVC(kernel='linear',shrinking=False)
ti = time.time()
clf2.fit(X, y)
print("Tiempo de entrenamiento: \n")
imprimir_tiempo(ti)

Tiempo de entrenamiento: 

--- 3.6378072023391725 mins ---


In [26]:
Xval, yval = data2Xy_bow(transf_with_stem, validation)
ti = time.time()
print("Accuracy: " + str(clf2.score(Xval, yval)))
print("Tiempo de validación: \n")
imprimir_tiempo(ti)

Accuracy: 0.9446838243856993
Tiempo de validación: 

--- 0.08876012563705445 mins ---


In [27]:
X_centroide_train,y_centroide_train = data2Xy_vec(vectors, train)

SVM_centroide2 = sklearn.svm.SVC(kernel='linear',shrinking=False)
ti = time.time()
SVM_centroide2.fit(X_centroide_train, y_centroide_train)
print("Tiempo de entrenamiento: \n")
imprimir_tiempo(ti)

Tiempo de entrenamiento: 

--- 2.4220492164293925 mins ---


In [28]:
X_centroide_val, y_centroide_val = data2Xy_vec(vectors, validation)
ti = time.time()
print("Accuracy: " + str(SVM_centroide2.score(X_centroide_val, y_centroide_val)))
print("Tiempo de validación: \n")
imprimir_tiempo(ti)

Accuracy: 0.9470129265168278
Tiempo de validación: 

--- 0.5272740046183269 mins ---


In [29]:
def printClassifierResult(classifier, X, y, modelText):
    predicted = classifier.predict(X)
    conf = sklearn.metrics.confusion_matrix(y, predicted)
    tp, tn, fp, fn = conf[1][1], conf[0][0], conf[0][1], conf[1][0]
    print("\nPara el modelo de {} se obtiene lo siguiente:".format(modelText))
    print("Hay {} verdaderos positivos, {} verdaderos negativos, {} falsos positivos, {} falsos negativos".format(tp, tn, fp, fn))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1Score = 2 * precision * recall / (precision + recall)
    print("Precision: {}, recall: {}, Fscore: {}".format(precision, recall, f1Score))

In [30]:
printClassifierResult(clf2, Xval, yval, "SVM con Bag of Words")
printClassifierResult(SVM_centroide2, X_centroide_val, y_centroide_val, "SVM con Centroides")


Para el modelo de SVM con Bag of Words se obtiene lo siguiente:
Hay 5448 verdaderos positivos, 2664 verdaderos negativos, 341 falsos positivos, 134 falsos negativos
Precision: 0.9410951805147694, recall: 0.9759942672877105, Fscore: 0.9582270688593792

Para el modelo de SVM con Centroides se obtiene lo siguiente:
Hay 5404 verdaderos positivos, 2728 verdaderos negativos, 277 falsos positivos, 178 falsos negativos
Precision: 0.9512409787009329, recall: 0.9681117878896452, Fscore: 0.9596022374145433


## Evaluación y análisis de resultados

### Precision, Recall y Matriz de Confusión

Calcule la medida de *accuracy* en el conjunto de *test* para los modelos de la parte anterior. Despliegue los resultados obtenidos.

In [31]:
Xtest, ytest = data2Xy_bow(transf_with_stem, test)
X_centroide_test,y_centroide_test = data2Xy_vec(vectors, test)

print("SVM bow tiene accuracy {} para el conjunto de test".format(clf.score(Xtest, ytest)))
print("FF NN bow tiene accuracy {} para el conjunto de test".format(clfNN.score(Xtest, ytest)))
print("SVM centroide tiene accuracy {} para el conjunto de test".format(SVM_centroide.score(X_centroide_test, y_centroide_test)))
print("FF NN centroide tiene accuracy {} para el conjunto de test".format(NN_centroide.score(X_centroide_test, y_centroide_test)))


SVM bow tiene accuracy 0.9466195977749251 para el conjunto de test
FF NN bow tiene accuracy 0.9473684210526315 para el conjunto de test
SVM centroide tiene accuracy 0.9603123662815576 para el conjunto de test
FF NN centroide tiene accuracy 0.9567821994009413 para el conjunto de test


Seleccione uno de los modelos y analice sus errores en función de la matriz de confusión. Compute las medidas de *precision*, *recall* y *F*.

In [32]:
printClassifierResult(clfNN, Xtest, ytest, "Feed Forward Neural Network con Bag of Words")


Para el modelo de Feed Forward Neural Network con Bag of Words se obtiene lo siguiente:
Hay 5972 verdaderos positivos, 2884 verdaderos negativos, 352 falsos positivos, 140 falsos negativos
Precision: 0.9443390259329538, recall: 0.9770942408376964, Fscore: 0.9604374396912191


Despliegue algunos casos de falsos positivos y falsos negativos del clasificador seleccionado.

In [33]:
predicted = clfNN.predict(Xtest)

print("######### Falsos positivos #########\n")

# falsos positivos
total_fp_printed = 0
for i in range(len(predicted)):
    if predicted[i] == 'POS' and predicted[i] != ytest[i]:
        print(test[i])
        total_fp_printed += 1
    if total_fp_printed == 10: 
        break;
        
print("\n######### Falsos negativos #########\n")

# falsos negativos
total_fn_printed = 0
for i in range(len(predicted)):
    if predicted[i] == 'NEG' and predicted[i] != ytest[i]:
        print(test[i])
        total_fn_printed += 1
    if total_fn_printed == 10: 
        break;

######### Falsos positivos #########

('Increiblemente fui con expectativas que me crearon seguidores de este restaurante y la verdad no fueron superadas. Probe llama, ñandu y yacaré y a las tres carnes les faltaba sabor; las presentaciones (?). El servicio flojo. Y sobre la ambientacion no me gusto, es una mezcla de "restaurante" y "almacen" quizas asi se refleja que no es ni una cosa ni la otra.', 'NEG')
('El lugar es muy lindo y la música también, pero la comida no tanto... comimos unos tacos de carne y de pescado, los primeros zafaban, pero los segundos casi no tenían pescado... cuando preguntamos por los postres, nos dijeron que tenían muchos pedidos en la cocina ¿? En resumen, para pasar un buen rato está bien, pero no me gustaron los tacos.', 'NEG')
('Somo italianos y la verdad que preferimos la pizza Napoletana en serio, como la de Siamo nel Forno por ejemplo, pero este lugar nos encantò, muy muy lindo. La calidad deja muchas dudas, la atenciòn ni hablar, muy ruidoso tambien cu

Al observar con detenimiento los falsos positivos y los falsos negativos, se puede ver en términos generales que los mismos son un tanto neutros, es decir, comentarios que tienen en su contenido una mención positiva, pero que al mismo tiempo como contraste hacen notar una característica negativa, creemos que es por esta razón que se clasificaron incorrecamente por el modelo elegido. 

Para una ejecución en particular -notar que solo imprimimos 20 *falsos* sin un orden preestablecido-, tomamos algunos comentarios para ejemplificar lo que mencionamos:

*('Excelente ambiente. El menu no tiene relación con el lugar. Caro y pretensioso.', 'NEG')*
- **Aspecto positivo:** el ambiente
- **Aspecto negativo:** el precio

Si tuvieramos que clasificarlo lo haríamos negativo porque tiene dos oraciones negativas contra una positiva, y la palabra caro es muy negativa en estos casos, pesando más que el lugar en si mismo.

*('El lugar está bien ambientado y al principio todos son muy amables. Fuimos a disfrutar del premio #mesadeamigos de guia oleo y casi se nos frustra el festejo por una falta de comunicación entre la gente del lugar. Solucionado el problema todo bien. Yo pedí un bife de chorizo y la carne estaba roja por dentro y quemada por fuera... las carnes no son lo más recomendable. Sí los postres que son muy ricos.', 'NEG')*
- **Aspecto positivo:** ambiente, amabilidad, los postres
- **Aspecto negativo:** falta de comunicación, mala cocción y calidad de las carnes

Es un comentario neutro a nuestro entender, dado que el festejo no salió frustrado finalmente, y hubo apenas un problema en la cocción de la carne.

*('el lugar excelente. un poquito lento el plato principal de sushi pero realmente excelente un abrazo a todos con el descuento de GO pagamos 180 por persona', 'POS')*
- **Aspecto positivo**: el lugar, el descuento
- **Aspecto negativo**: lento el plato principal

Claramente un comentario positivo, suponemos que la palabra lento sucede en muchos comentarios anotados negativamente y eso hizo pesar en la predicción.

*('Muy buen ambiente, moderno y con buen gusto. Fuimos el jueves 23 a las 20.15 y no habia demasiada gente, quizas tardaron en servir los platos pero la amabilidad estuvo bien.', 'POS')*
- **Aspecto positivo**: ambiente, amabilidad
- **Aspecto negativo**: demora habiendo poca gente

Nuevamente un comentario positivo a nuestro entender, quizá la palabra *tardaron* debe ser una palabra con mucho peso negativo en este contexto.

### Muestreo de oraciones

Utilice cada uno de los clasificadores considerados anteriormente para cada comentario de *lista_comentarios* y *lista_comentarios_estudiante*.

Despliegue en pantalla cada comentario junto con la salida obtenida por cada clasificador. Realice los comentarios que considere pertinentes.

In [34]:
comentarios = [y for x in ([c1, c2] for c1,c2 in lista_comentarios + lista_comentarios_estudiante) for y in x]

for comentario in comentarios:
    X_comment = transf_with_stem.transform([comentario])
    X_centroide = txt2vec(vectors, comentario)
    print()
    print("'{}' fue clasificado como: \n'{}', '{}', '{}', '{}' \npara BOW con NN, BOW con SVM, centroide de vectores con NN y centroide de vectores con SVM respectivamente".format(comentario, clfNN.predict(X_comment)[0], clf.predict(X_comment)[0], NN_centroide.predict([X_centroide])[0], SVM_centroide.predict([X_centroide])[0]))
    
    


'muy rica la comida, buenas pizzas' fue clasificado como: 
'POS', 'POS', 'POS', 'POS' 
para BOW con NN, BOW con SVM, centroide de vectores con NN y centroide de vectores con SVM respectivamente

'excelente pizza la salsa estaba muy rica' fue clasificado como: 
'POS', 'POS', 'POS', 'POS' 
para BOW con NN, BOW con SVM, centroide de vectores con NN y centroide de vectores con SVM respectivamente

'no me gustó para nada. mala atención' fue clasificado como: 
'NEG', 'NEG', 'NEG', 'NEG' 
para BOW con NN, BOW con SVM, centroide de vectores con NN y centroide de vectores con SVM respectivamente

'me pareció todo bastante malo' fue clasificado como: 
'NEG', 'NEG', 'NEG', 'NEG' 
para BOW con NN, BOW con SVM, centroide de vectores con NN y centroide de vectores con SVM respectivamente

'que buen servicio, hay que volver' fue clasificado como: 
'POS', 'POS', 'POS', 'POS' 
para BOW con NN, BOW con SVM, centroide de vectores con NN y centroide de vectores con SVM respectivamente

'excelente todo, m

De los 16 comentarios clasificados, en 14 de ellos los cuatro modelos coincideron en la clasificación. Es decir que solo hubo disconcordancia en el 13% de los comentarios (2 de 16).

Creemos que la clasificación para los 14 comentarios es correcta, dado que en ellos hay una clara connotación ya sea negativa o positiva pero no ambas a la vez, es decir, no los consideramos comentarios neutros.

Por otro lado observamos los comentarios para los cuales no hubo unanimidad en su clasificación:

**'las tartas no me gustaron' fue clasificado como: 
'POS', 'POS', 'NEG', 'NEG' 
para BOW con NN, BOW con SVM, centroide de vectores con NN y centroide de vectores con SVM respectivamente**

A nuestro entender el comentario es claro y conciso, definitivamente negativo. Dado esto, los modelos que usaron centroide de vectores acertaron, mientras que los modelos que usaron representación BOW fallaron en su clasificación.

Notar que para este caso *gustaron* y en particular su stem *gust* es la característica que se utilizó para clasificar. Notar que la palabra *no* revierte la connotación que tiene el stem gust, sin embargo no es considerada para la clasificación, ya que no se encuentra en el vocabulario, seguramente por haber sido filtrada al utilizar los parametros max_df y min_df.

**'Guijón: Por favor no cambien nunca. Gracias' fue clasificado como: 
'NEG', 'POS', 'POS', 'POS' 
para BOW con NN, BOW con SVM, centroide de vectores con NN y centroide de vectores con SVM respectivamente**

Nuevamente el comentario es claramente positivo, los modelos con representación de centroide nuevamente clasificaron correctamente, pero esta vez solo falló la red neuronal utilizando representación BOW.

A diferencia con el comentario anterior, la mala clasificación radica en el modelo en si mismo y no tanto en la   representación vectorial utilizada.

Creemos que en ambos casos la representación de centroides se ve favorecida en que sus vectores tienen mayor información ya que tienen contenido en si mismo el contexto (por utilizar vectores de palabras), lo cual hace que los vectores resultantes de cada comentario sean más representativos aportando mayor información sobre el tipo de contenido del mismo. Por otro lado, el modelo BOW solo tiene en cuenta la cantidad de ocurrencias de las palabras en un documento sobre un vocabulario conocido, creemos que en comentarios relativamente cortos y con palabras nuevas dentro del vocabulario, no contendran la suficiente información para poder clasificarlos correctamente. Esto no hace más que reconfirmar los resultados obtenidos en secciones anteriores, donde el F1 score fue mejor para los modelos que usaron representación de centroides.